In [ ]:
# # 워드클라우드 라이브러리 설치
# !pip install wordcloud

# # Mecab 설치 (Colab)
# !pip install konlpy
# !pip install mecab-python3
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
# !pip install -q -U transformers bitsandbytes accelerate peft # "런타임 다시 시작"

# 📊 Phase 1용 데이터셋 (8개)


# **데이터 수집**

In [ ]:
from google.colab import drive
import pandas as pd
import json


# 구글 드라이브 마운트
drive.mount('/content/drive')


In [ ]:
save_path='/content/drive/MyDrive/태혁/data/'
#save_path = '/content/drive/MyDrive/프로젝트/2025_과외_프로젝트1/'


In [ ]:



raw_data = pd.read_excel(save_path+'data/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.xlsx')
raw_data

In [ ]:
raw_data.info()

In [ ]:
raw_data['연령'].unique()

In [ ]:
data_path=save_path+'data/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training.json'

with open(data_path, 'r', encoding='utf-8') as f:
    data=json.load(f)
    label_data= pd.json_normalize(data)
    label_data.columns = label_data.columns.str.replace('.', '_', regex=False)


In [ ]:
label_data

In [ ]:
len(raw_data), len(label_data)

## 데이터 확인 label데이터 필요성 없다 확인

In [ ]:
raw_data=raw_data.drop('Unnamed: 0', axis=1)

In [ ]:
teen_data=raw_data[raw_data['연령']=='청소년']

In [ ]:
teen_data.columns[2:6]

In [ ]:
for i in teen_data.columns[2:6]:

    print('---------------------------------------')
    print(i)
    print(teen_data[i].value_counts())
    print('---------------------------------------')
    print()
    print()

In [ ]:
teen_data[:10]

In [ ]:
data=teen_data.drop(columns=['연령','성별','상황키워드','신체질환','시스템문장1','시스템문장2','시스템문장3'])

In [ ]:
data

# **데이터 전처리**

## 사람문장 전체 텍스트

In [ ]:
all_text=data[['사람문장1','사람문장2','사람문장3']]

In [ ]:
all_text=all_text.apply(lambda x: " ".join(x.dropna()), axis=1).reset_index(drop=True)

In [ ]:
all_text.name='text'

In [ ]:
all_text

##사람문장 첫번째 텍스트

In [ ]:
first_text=data['사람문장1'].reset_index(drop=True)

In [ ]:
first_text.name='text'

In [ ]:
first_text

## 대분류 6개 감정 라벨

In [ ]:
six_label=data['감정_대분류'].reset_index(drop=True)

## 소분류 4개 감정 라벨

In [ ]:
small_label=data['감정_소분류']

In [ ]:
depression_keywords_list = [
    '우울한', '좌절한', '버려진', '비통한', '염세적인', '마비된', '고립된',
    '외로운', '한심한', '죄책감의', '괴로워하는', '취약한', '가난한, 불우한',
    '희생된', '낙담한', '혼란스러운'
]

In [ ]:
positive_keywords_list = [
    '감사하는', '사랑하는', '편안한', '만족스러운', '흥분되는', '느긋한',
    '안도하는', '신이 난', '자신하는', '신뢰하는', '기쁨', '흥분', '안도', '느긋'
]


In [ ]:
general_negative_keywords_list = [
    '툴툴대는', '실망한', '두려운', '질투하는', '스트레스 받는', '배신당한',
    '짜증내는', '후회되는', '충격 받은', '악의적인', '당혹스러운', '열등감',
    '안달하는', '구역질 나는', '회의적인', '눈물이 나는', '걱정스러운',
    '부끄러운', '억울한', '노여워하는', '조심스러운', '혐오스러운',
    '성가신', '환멸을 느끼는', '초조한', '분노', '불안', '슬픔', '상처', '당황'
]


In [ ]:
def labeling(x):
    if x in depression_keywords_list:
        return "우울 신호"

    elif x in positive_keywords_list:
        return "긍정"

    elif x in general_negative_keywords_list:
        return "일반 부정"

    else:
        return "중립"

In [ ]:
four_label=pd.Series([labeling(i) for i in small_label])

In [ ]:
four_label.name='감정 분류'

In [ ]:
len(four_label)

## **텍스트정제**

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ가-힣.,?! ]','',text)  #한글, 영어 소문자, 숫자, 그리고 기본적인 문장 부호(. , ? !)를 제외한 모든 문자 제거
    text = text.strip()  # 불필요한 공백 처리# 문장 앞뒤의 공백을 제거합니다.
    text= re.sub(r'\s+', ' ', text) # 여러 개의 공백을 하나의 공백으로 바꿉니다

    return text


In [ ]:
df=pd.DataFrame()

In [ ]:
all_text =all_text.apply(lambda x: clean_text(x))

In [ ]:
first_text =first_text.apply(lambda x: clean_text(x))

In [ ]:
len(all_text), len(first_text), len(six_label), len(four_label)

# **EDA**

### 한글 폰트

In [ ]:
# 코랩 #런타임 다시시작 안해도 댐

import matplotlib
from matplotlib import font_manager

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv

fontpaths = ["/usr/share/fonts/truetype/nanum/"]
font_files = font_manager.findSystemFonts(fontpaths=fontpaths)

for ff in font_files:
    font_manager.fontManager.addfont(ff)

matplotlib.rc('font', family="NanumGothic")
matplotlib.rcParams['axes.unicode_minus'] = False

## 클래스 분포 분석

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
six_label.value_counts()

In [ ]:
sns.barplot(six_label.value_counts())

In [ ]:
four_label.value_counts()

In [ ]:
sns.barplot(four_label.value_counts())

## max_length 찾기

In [ ]:
from transformers import AutoTokenizer
import numpy as np

In [ ]:
# Klue 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

In [ ]:
all_text_token_length=[]

In [ ]:
for text in all_text:
    encoding=tokenizer(text, add_special_tokens=True)
    all_text_token_length.append(len(encoding['input_ids']))

print(f'총 {len(all_text_token_length)}개의 문장에 대한 토큰 길이 계산 완료')
print(f'예시 앞 5개 문장길이 {all_text_token_length[:5]}')

In [ ]:
figure, axis = plt.subplots()
sns.histplot(all_text_token_length, kde=True, bins=50, ax=axis)
axis.set_ylabel('Count')
axis.set_xlabel('Token Length')
axis.set_title('ALL_TEXT token distribution')

In [ ]:
print('모든 문장에서')
print(f'토큰 길이 최댓값은 {max(all_text_token_length)} 이다.')
print(f'토큰 길이 최솟값은 {min(all_text_token_length)} 이다.')

print(f'토큰 길이 평균은 {np.mean(all_text_token_length)} 이다.')
print(f'토큰 길이 표준편차는 {np.std(all_text_token_length)} 이다.')


print("-" * 30)
print(f"90% 지점의 길이: {np.percentile(all_text_token_length, 90)}")
print(f"95% 지점의 길이: {np.percentile(all_text_token_length, 95)}")
print(f"99% 지점의 길이: {np.percentile(all_text_token_length, 99)}")

In [ ]:
all_text_max_length=np.percentile(all_text_token_length, 90)
print(all_text_max_length)

----
----

In [ ]:
first_text_token_length=[]

In [ ]:
for text in first_text:
    encoding=tokenizer(text, add_special_tokens=True)
    first_text_token_length.append(len(encoding['input_ids']))

print(f'총 {len(first_text_token_length)}개의 문장에 대한 토큰 길이 계산 완료')
print(f'예시 앞 5개 문장길이 {first_text_token_length[:5]}')

In [ ]:
figure, axis = plt.subplots()
sns.histplot(first_text_token_length, kde=True, bins=50, ax=axis)
axis.set_ylabel('Count')
axis.set_xlabel('Token Length')
axis.set_title('FIRST_TEXT token distribution')

In [ ]:
print('첫 문장에서')
print(f'토큰 길이 최댓값은 {max(first_text_token_length)} 이다.')
print(f'토큰 길이 최솟값은 {min(first_text_token_length)} 이다.')

print(f'토큰 길이 평균은 {np.mean(first_text_token_length)} 이다.')
print(f'토큰 길이 표준편차는 {np.std(first_text_token_length)} 이다.')


print("-" * 30)
print(f"90% 지점의 길이: {np.percentile(first_text_token_length, 90)}")
print(f"95% 지점의 길이: {np.percentile(first_text_token_length, 95)}")
print(f"99% 지점의 길이: {np.percentile(first_text_token_length, 99)}")

In [ ]:
first_text_max_length=np.percentile(first_text_token_length, 90)
print(first_text_max_length)

## 워드클라우드

In [ ]:

# 라이브러리 임포트
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd

from konlpy.tag import Mecab

# Mecab 객체 생성
mecab = Mecab()

In [ ]:
# 형태소 분석을 통해 문장들을 의미있는 단어 단위 들로 쪼개준다.

def tokenize_and_filter(corpus):

    # 1.형태소 분석 (품사 태깅)
    tokens_with_pos = mecab.pos(corpus)

    # 2.원하는 품사의 단어만 필터링
    meaningful_tags = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'MAG']

    # 3.길이가 2 이상인 단어만 추출 (한 글자 단어는 노이즈가 많음)
    filtered_words = [word for word, tag in tokens_with_pos if tag in meaningful_tags and len(word) > 1]

    return filtered_words

### 6개 라벨에 따른 워드클라우드

In [ ]:
six_label.value_counts()

In [ ]:
six_label_wordcloud=pd.concat([all_text, six_label], axis=1)

In [ ]:
anxiety_corpus = ' '.join(six_label_wordcloud[six_label_wordcloud['감정_대분류'] == '불안']['text'])
sadness_corpus = ' '.join(six_label_wordcloud[six_label_wordcloud['감정_대분류'] == '슬픔']['text'])
embarrassment_corpus = ' '.join(six_label_wordcloud[six_label_wordcloud['감정_대분류'] == '당황']['text'])
hurt_corpus = ' '.join(six_label_wordcloud[six_label_wordcloud['감정_대분류'] == '상처']['text'])
anger_corpus = ' '.join(six_label_wordcloud[six_label_wordcloud['감정_대분류'] == '분노']['text'])
joy_corpus = ' '.join(six_label_wordcloud[six_label_wordcloud['감정_대분류'] == '기쁨']['text'])

In [ ]:
anxiety_tokens = tokenize_and_filter(anxiety_corpus)
sadness_tokens = tokenize_and_filter(sadness_corpus)
embarrassment_tokens = tokenize_and_filter(embarrassment_corpus)
hurt_tokens = tokenize_and_filter(hurt_corpus)
anger_tokens = tokenize_and_filter(anger_corpus)
joy_tokens = tokenize_and_filter(joy_corpus)

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def wordcloud_tokens(tokens, title):

    word_counts=Counter(tokens)

    top_100_words=word_counts.most_common(100)


    # 워드클라우드 객체 생성 (generate 대신 generate_from_frequencies 사용)
    font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

    wordcloud=WordCloud(
        font_path=font_path,
        width=800,
        height=400,
        background_color='white'
    ).generate_from_frequencies(dict(top_100_words))

    # 시각화
    plt.figure(figsize=(15, 7))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, size=20)
    plt.axis('off')
    plt.show()



In [ ]:
wordcloud_tokens(anxiety_tokens, '불안')

In [ ]:
wordcloud_tokens(sadness_tokens, '슬픔')

In [ ]:
wordcloud_tokens(embarrassment_tokens, '당황')

In [ ]:
wordcloud_tokens(hurt_tokens, '상처')

In [ ]:
wordcloud_tokens(anger_tokens, '분노')

In [ ]:
wordcloud_tokens(joy_tokens, '기쁨')

### 4개 라벨에 따른 워드클라우드

In [ ]:
four_label.value_counts()

In [ ]:
four_label_wordcloud=pd.concat([all_text, four_label], axis=1)

In [ ]:
depression_corpus = ' '.join(four_label_wordcloud[four_label_wordcloud['감정 분류'] == '우울 신호']['text'])
negative_corpus = ' '.join(four_label_wordcloud[four_label_wordcloud['감정 분류'] == '일반 부정']['text'])
positive_corpus = ' '.join(four_label_wordcloud[four_label_wordcloud['감정 분류'] == '긍정']['text'])
neutral_corpus = ' '.join(four_label_wordcloud[four_label_wordcloud['감정 분류'] == '중립']['text'])

In [ ]:
depression_tokens = tokenize_and_filter(depression_corpus)
negative_tokens = tokenize_and_filter(negative_corpus)
positive_tokens = tokenize_and_filter(positive_corpus)
neutral_tokens = tokenize_and_filter(neutral_corpus)

In [ ]:
wordcloud_tokens(depression_tokens, '우울_신호')

In [ ]:
wordcloud_tokens(negative_tokens, '일반_부정')

In [ ]:
wordcloud_tokens(positive_tokens, '긍정')

In [ ]:
wordcloud_tokens(neutral_tokens, '중립')

## 데이터 준비

In [ ]:
six_label_all_text=pd.concat([all_text,six_label], axis=1)

In [ ]:
six_label_all_text.to_csv(save_path + 'six_label_all_text.csv', index=False)

In [ ]:
six_label_all_text

In [ ]:
four_label_all_text=pd.concat([all_text,four_label], axis=1)

In [ ]:
four_label_all_text.to_csv(save_path + 'four_label_all_text.csv', index=False)

In [ ]:
four_label_all_text

In [ ]:
six_label_first_text=pd.concat([first_text,six_label], axis=1)

In [ ]:
six_label_first_text.to_csv(save_path + 'six_label_first_text.csv', index=False)

In [ ]:
six_label_first_text

In [ ]:
four_label_first_text=pd.concat([first_text,four_label], axis=1)

In [ ]:
four_label_first_text.to_csv(save_path + 'four_label_first_text.csv', index=False)

In [ ]:
four_label_first_text

# **모델링(+토큰화,임베딩)**

## **MLP**

토큰화 => 임베딩 => 모델링



### 임베딩

### 0.데이터 분활 및 로드

TensorDataset => train_test_split => DataLoader

### 1.모델 정의
- 데이터의 종류와 해결하려는 문제에 따라 모델의 구조, 특히 모델 구조 설계 단계가 크게 달라집니다

### 2.모델 컴파일/ 학습 설정
- 손실 함수(Loss Function), 옵티마이저(Optimizer), 성능 평가지표(Metrics)//경사하강법,오차역전파



### 3.모델 학습/ 학습 루프
- 순전파(학습) => 손실계산 => 역전파(그래디언트 계산:가중치를 얼마나 조정할지?) => 옵티마이저 업데이트(가중치 업데이트)

### 4.모델 평가
-model.eval() - 평가 모드로 전환, torch.no_grad() - 그래디언트 계산 안 함
-torch.argmax(y_pred, dim=1) ==> dim=1은 (32, 6)이라면 6을 말함

- **평가지표**

## **Fine_Tuning**

### 0.데이터 분활 및 로드

- TensorDataset => train_test_split => DataLoader
- FIne_Tuning에서는 train_test_split => Dataset 클래스 => DataLoader
- train_test_split 이후 인덱스 리셋하기

### 1.모델정의

### 2.모델 컴파일/ 학습 설정
"- 손실 함수(Loss Function), 옵티마이저(Optimizer), 성능 평가지표(Metrics)//경사하강법,오차역전파"



### 3.모델 학습/ 학습 루프
- 순전파(학습) => 손실계산 => 역전파(그래디언트 계산:가중치를 얼마나 조정할지?) => 옵티마이저 업데이트(가중치 업데이트)


### 4.모델 평가
-model.eval() - 평가 모드로 전환, torch.no_grad() - 그래디언트 계산 안 함
-torch.argmax(y_pred, dim=1) ==> dim=1은 (32, 6)이라면 6을 말함

- **평가지표**

# **함수화**

## MLP

### 데이터로드 함수

In [ ]:
# max_length 설정

all_text_max_length, first_text_max_length

In [ ]:
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split



batch_size=32

In [ ]:

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts.reset_index(drop=True)   # 확실하게 리셋
        self.labels = labels.reset_index(drop=True) # 확실하게 리셋
        self.tokenizer=tokenizer
        self.max_length=int(max_length)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = int(self.labels.iloc[idx])

        encoding=self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
             )
        return (
                encoding['input_ids'].flatten(),
                encoding['attention_mask'].flatten(),
                torch.tensor(label, dtype=torch.long)
        )

In [ ]:
def MLP_data_load(text, max_length):

    tokenizer=AutoTokenizer.from_pretrained("klue/roberta-base")


    label_encoder=LabelEncoder()
    label_encoded=label_encoder.fit_transform(text.iloc[:,1])
    label_count=text.iloc[:,1].nunique()

    print(f"클래스 수: {label_count}")
    for idx, label_name in enumerate(label_encoder.classes_):
        print(f"  {idx}: {label_name}")


    label_encoded_series = pd.Series(label_encoded, name='label_encoded')
    df=pd.concat([text['text'].reset_index(drop=True), label_encoded_series],axis=1)


    X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['label_encoded'],
    test_size=0.3,
    stratify=df['label_encoded'],
    random_state=42
    )


    X_train=X_train.reset_index(drop=True)
    X_test=X_test.reset_index(drop=True)
    y_train=y_train.reset_index(drop=True)
    y_test=y_test.reset_index(drop=True)



    train_dataset = TextDataset(
    texts=X_train,
    labels=y_train,
    tokenizer=tokenizer,
    max_length=max_length
    )

    test_dataset = TextDataset(
        texts=X_test,
        labels=y_test,
        tokenizer=tokenizer,
        max_length=max_length
    )

    train_loader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    print(f"✅ 훈련 데이터: {len(train_dataset)}개")
    print(f"✅ 테스트 데이터: {len(test_dataset)}개")

    return train_loader, test_loader, label_count


### 모델 함수

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm


In [ ]:
class MLP_model(nn.Module):

    def __init__(self, hid_dim, out_dim, p=0.2):
        super().__init__()

        klue_model = AutoModel.from_pretrained('klue/roberta-base')
        self.embeddings = klue_model.embeddings    # ⭐ 임베딩 레이어만 추출


        # 임베딩 레이어 고정 (학습 안 함)
        for param in self.embeddings.parameters():
            param.requires_grad = False


        self.layer=nn.Sequential(
                nn.Linear(768, hid_dim),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(hid_dim, hid_dim//2),
                nn.ReLU(),
                nn.Dropout(p),
                nn.Linear(hid_dim//2, out_dim)
        )


    def forward(self, input_ids, attention_mask):

        embedding_output = self.embeddings(input_ids=input_ids)

        # 2. Attention mask를 고려한 평균 풀링
        mask_expanded = attention_mask.unsqueeze(-1).expand(embedding_output.size()).float()
        sum_embeddings = torch.sum(embedding_output * mask_expanded, 1)
        sum_mask = mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)  # 0으로 나누기 방지
        mean_embeddings = sum_embeddings / sum_mask
        # 출력 shape: (batch_size, 768)


        return self.layer(mean_embeddings)

In [ ]:
def MLP_train(train_loader, model, criterion, optimizer, device=device, epochs=epochs):

    history = {'Epoch': [], 'loss': []}

    for i in range(epochs):

        model.train()
        total_loss=0

        for x, attention_mask, y in tqdm(train_loader, desc=f"Epoch {i+1}/{epochs}"):
            x = x.to(device)
            attention_mask = attention_mask.to(device)
            y = y.to(device)


            y_pred=model(x, attention_mask) # 순전파

            loss=criterion(y_pred, y) # 손실계산

            optimizer.zero_grad()  # 역전파
            loss.backward()

            optimizer.step() # 옵티마이저 업데이트


            total_loss += loss.item()

        history['loss'].append(total_loss / len(train_loader))
        history['Epoch'].append(i)

        if i % 2 == 0:
            print (f'Epoch {i}: Loss={total_loss/len(train_loader)}')

    return history

### 모델 평가 함수

In [ ]:
    from sklearn.metrics import(
        accuracy_score,
        f1_score,
        precision_score,
        recall_score,
        classification_report,
        confusion_matrix
    )
    import numpy as np

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
def MLP_eval(test_loader, model, depression_classes, device=device, label_names=None ):
    model.eval()

    all_preds=[]
    all_labels=[]

    with torch.no_grad():
        for x, attention_mask, y in test_loader:
            x = x.to(device)
            attention_mask = attention_mask.to(device)

            y_pred=model(x,attention_mask)
            preds=torch.argmax(y_pred, dim=1)


            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())



    accuracy = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    print(f"\n✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Macro F1-Score: {f1_macro:.4f}")
    print(f"✅ Weighted F1-Score: {f1_weighted:.4f}")
    print(f"\n🔢 Confusion Matrix:")
    print(cm)

    # ============================================
    # 3. 우울 신호 집중 분석
    # ============================================

    print("\n" + "="*60)
    print("🎯 우울 신호 집중 분석 (불안 + 상처 + 슬픔)")
    print("="*60)

    # 이진 분류로 변환
    all_labels_np = np.array(all_labels)
    all_preds_np = np.array(all_preds)

    y_true_binary = np.isin(all_labels_np, depression_classes).astype(int)
    y_pred_binary = np.isin(all_preds_np, depression_classes).astype(int)

    # Precision & Recall
    precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    f1 = f1_score(y_true_binary, y_pred_binary, zero_division=0)



    # 실제 우울 신호인데 기타로 예측
    false_negatives = np.sum((y_true_binary == 1) & (y_pred_binary == 0))
    # True Positive
    true_positives = np.sum((y_true_binary == 1) & (y_pred_binary == 1))
    # 전체 우울 신호
    total_depression = np.sum(y_true_binary == 1)



    print(f"\n{'='*60}")
    print("🎯 우울 신호 집중 분석 (불안 + 상처 + 슬픔)")
    print(f"{'='*60}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1-Score: {f1:.4f}")
    print(f"\n⚠️ False Negative: {false_negatives}건 / {total_depression}건")
    print(f"✅ True Positive: {true_positives}건 / {total_depression}건")


    # Classification Report
    if label_names: # = ['기쁨', '당황', '분노', '불안', '상처', '슬픔']
        print(f"\n📋 Classification Report:")
        print(classification_report(all_labels, all_preds, target_names=label_names))



    return {
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'depression_precision': precision,
        'depression_recall': recall,
        'depression_f1': f1,
        'false_negatives': false_negatives,
        'true_positives': true_positives,
        'total_depression': total_depression,
    }


## **Fine_Tuning**

### 데이터로드 함수

In [ ]:
# max_length 설정

all_text_max_length, first_text_max_length

In [ ]:
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


batch_size=32

In [ ]:

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts.reset_index(drop=True)   # 확실하게 리셋
        self.labels = labels.reset_index(drop=True) # 확실하게 리셋
        self.tokenizer=tokenizer
        self.max_length=int(max_length)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = int(self.labels.iloc[idx])

        encoding=self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
             )
        return (
                encoding['input_ids'].flatten(),
                encoding['attention_mask'].flatten(),
                torch.tensor(label, dtype=torch.long)
        )

In [ ]:
def FineTuning_data_load(text, max_length):

    tokenizer=AutoTokenizer.from_pretrained('klue/roberta-base')


    label_encoder=LabelEncoder()
    label_encoded=label_encoder.fit_transform(text.iloc[:,1])
    label_count=text.iloc[:,1].nunique()

    print(f"클래스 수: {label_count}")
    for idx, label_name in enumerate(label_encoder.classes_):
        print(f"  {idx}: {label_name}")


    label_encoded_series = pd.Series(label_encoded, name='label_encoded')
    df=pd.concat([text['text'].reset_index(drop=True), label_encoded_series],axis=1)


    X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['label_encoded'],
    test_size=0.3,
    stratify=df['label_encoded'],
    random_state=42
    )


    X_train=X_train.reset_index(drop=True)
    X_test=X_test.reset_index(drop=True)
    y_train=y_train.reset_index(drop=True)
    y_test=y_test.reset_index(drop=True)



    train_dataset = TextDataset(
    texts=X_train,
    labels=y_train,
    tokenizer=tokenizer,
    max_length=max_length
    )

    test_dataset = TextDataset(
        texts=X_test,
        labels=y_test,
        tokenizer=tokenizer,
        max_length=max_length
    )

    train_loader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    print(f"✅ 훈련 데이터: {len(train_dataset)}개")
    print(f"✅ 테스트 데이터: {len(test_dataset)}개")

    return train_loader, test_loader, label_count


### 모델 함수

In [ ]:
from transformers import AutoModel,AutoConfig
import torch.nn as nn
from tqdm import tqdm


In [ ]:
class klueClassifier(nn.Module):
    def __init__(self, num_class, num_layers_to_train=3, dropout=0.3):
        super(klueClassifier,self).__init__()

        self.klue=AutoModel.from_pretrained('klue/roberta-base')
        config= AutoConfig.from_pretrained('klue/roberta-base')
        hidden_size=config.hidden_size

        # ============================================
        # 부분 파인튜닝
        total_layers = 12  # klue는 12개 레이어
        layers_to_freeze = total_layers - num_layers_to_train

        for param in self.klue.embeddings.parameters():
            param.requires_grad = False

        # 2) 처음 N개 레이어 freeze
        for i in range(layers_to_freeze):
            for param in self.klue.encoder.layer[i].parameters():
                param.requires_grad = False
        # ============================================


        self.dropout=nn.Dropout(dropout)
        self.classifier= nn.Linear(hidden_size, num_class)



    def forward(self, input_ids, attention_mask):

        output=self.klue(input_ids=input_ids, attention_mask=attention_mask)

        cls_token=output.pooler_output

        x=self.dropout(cls_token)
        y_pred=self.classifier(x)

        return y_pred



In [ ]:

def FineTuning_train(train_loader, model, criterion, optimizer, device=device, epochs=epochs):

    history = {'Epoch': [], 'loss': []}

    for i in range(epochs):
        model.train()
        total_loss=0

        for x,attention_mask,y in tqdm(train_loader):

            x= x.to(device)
            attention_mask=attention_mask.to(device)
            y= y.to(device)

            y_pred=model(x, attention_mask)
            loss=criterion(y_pred, y)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            total_loss+=loss.item()


        history['loss'].append(total_loss / len(train_loader))
        history['Epoch'].append(i)

        if i % 2 == 0:
            print (f'Epoch {i}: Loss={total_loss/len(train_loader)}')

    return history


### 모델평가 함수

In [ ]:
    from sklearn.metrics import(
        accuracy_score,
        f1_score,
        precision_score,
        recall_score,
        classification_report,
        confusion_matrix
    )
    import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def FineTuning_eval(test_loader, model, depression_classes, device=device, label_names=None ):
    model.eval()

    all_preds=[]
    all_labels=[]

    with torch.no_grad():
        for x, attention_mask, y in test_loader:  # ✅ 3개!
            x = x.to(device)
            attention_mask = attention_mask.to(device)

            y_pred = model(x, attention_mask)  # ✅ attention_mask 포함
            preds = torch.argmax(y_pred, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())



    accuracy = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    print(f"\n✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Macro F1-Score: {f1_macro:.4f}")
    print(f"✅ Weighted F1-Score: {f1_weighted:.4f}")
    print(f"\n🔢 Confusion Matrix:")
    print(cm)



    # ============================================
    # 3. 우울 신호 집중 분석
    # ============================================


    print("\n" + "="*60)
    print(f"🎯 우울 신호 집중 분석 ({depression_classes})")
    print("="*60)


    # 이진 분류로 변환
    all_labels_np = np.array(all_labels)
    all_preds_np = np.array(all_preds)

    y_true_binary = np.isin(all_labels_np, depression_classes).astype(int)
    y_pred_binary = np.isin(all_preds_np, depression_classes).astype(int)

    # Precision & Recall
    precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
    recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
    f1 = f1_score(y_true_binary, y_pred_binary, zero_division=0)



    # 실제 우울 신호인데 기타로 예측
    false_negatives = np.sum((y_true_binary == 1) & (y_pred_binary == 0))
    # True Positive
    true_positives = np.sum((y_true_binary == 1) & (y_pred_binary == 1))
    # 전체 우울 신호
    total_depression = np.sum(y_true_binary == 1)



    print(f"\n{'='*60}")
    print("🎯 우울 신호 집중 분석 (불안 + 상처 + 슬픔)")
    print(f"{'='*60}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1-Score: {f1:.4f}")
    print(f"\n⚠️ False Negative: {false_negatives}건 / {total_depression}건")
    print(f"✅ True Positive: {true_positives}건 / {total_depression}건")


    # Classification Report
    if label_names: # = ['기쁨', '당황', '분노', '불안', '상처', '슬픔']
        print(f"\n📋 Classification Report:")
        print(classification_report(all_labels, all_preds, target_names=label_names))



    return {
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'depression_precision': precision,
        'depression_recall': recall,
        'depression_f1': f1,
        'false_negatives': false_negatives,
        'true_positives': true_positives,
        'total_depression': total_depression,
    }


# **six_label_all_text**

In [ ]:
total_result=[]

In [ ]:
results={'task':'six_label_all_text_mlp',
         'text': 'all',
         'label':'six',
         'model':'mlp'
                    }

In [ ]:
max_length=all_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=MLP_data_load(six_label_all_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mlp_model = MLP_model( hid_dim=512, out_dim=label_count, p=0.3).to(device)

mlp_criterion=nn.CrossEntropyLoss()
mlp_optimizer=torch.optim.AdamW(mlp_model.parameters(),lr=2e-3, weight_decay=1e-4)

In [ ]:
MLP_train(train_loader, mlp_model, mlp_criterion, mlp_optimizer, device, epochs=6)

In [ ]:
torch.save(mlp_model.state_dict(), save_path + 'phase1_six_label_all_text_mlp.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=MLP_eval(test_loader, mlp_model, six_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
import gc

del mlp_model      # 모델 삭제
del mlp_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache() # GPU 캐시 비우기
gc.collect()       # RAM 비우기
print("메모리 정리가 완료되었습니다.")

--------------------------------------------------------------------
##
--------------------------------------------------------------------


In [ ]:
results={'task':'six_label_all_text_finetuning',
         'text': 'all',
         'label':'six',
         'model':'finetuning'
                    }

In [ ]:
max_length=all_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=FineTuning_data_load(six_label_all_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
finetuning_model = klueClassifier(num_class=label_count, num_layers_to_train=3, dropout=0.3).to(device)


FineTuning_criterion=nn.CrossEntropyLoss()
FineTuning_optimizer=torch.optim.AdamW([{'params':finetuning_model.klue.parameters(), 'lr': 1e-4},
                             {'params':finetuning_model.classifier.parameters(), 'lr': 1e-4}
                             ], weight_decay=0.01)

In [ ]:
FineTuning_train(train_loader, finetuning_model, FineTuning_criterion, FineTuning_optimizer, device, epochs=6)

In [ ]:
torch.save(finetuning_model.state_dict(), save_path + 'phase1_six_label_all_text_finetuning.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=FineTuning_eval(test_loader, finetuning_model, six_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
total_result

In [ ]:
import gc
del finetuning_model      # 모델 삭제 (변수명 주의)
del FineTuning_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache()
gc.collect()
print("메모리 정리가 완료되었습니다.")

# **four_label_all_text**

In [ ]:
results={'task':'four_label_all_text_mlp',
         'text': 'all',
         'label':'four',
         'model':'mlp'
                    }

In [ ]:
four_label_all_text

In [ ]:
max_length=all_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=MLP_data_load(four_label_all_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mlp_model = MLP_model( hid_dim=512, out_dim=label_count, p=0.3).to(device)

mlp_criterion=nn.CrossEntropyLoss()
mlp_optimizer=torch.optim.AdamW(mlp_model.parameters(),lr=2e-3, weight_decay=1e-4)

In [ ]:
MLP_train(train_loader, mlp_model, mlp_criterion, mlp_optimizer, device, epochs=6)

In [ ]:
torch.save(mlp_model.state_dict(), save_path + 'phase1_four_label_all_text_mlp.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=MLP_eval(test_loader, mlp_model, four_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
import gc
del mlp_model      # 모델 삭제
del mlp_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache() # GPU 캐시 비우기
gc.collect()       # RAM 비우기
print("메모리 정리가 완료되었습니다.")

--------------------------------------------------------------------
##
--------------------------------------------------------------------


In [ ]:
results={'task':'four_label_all_text_finetuning',
         'text': 'all',
         'label':'four',
         'model':'finetuning'
                    }

In [ ]:
max_length=all_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=FineTuning_data_load(four_label_all_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
finetuning_model = klueClassifier(num_class=label_count, num_layers_to_train=3, dropout=0.3).to(device)


FineTuning_criterion=nn.CrossEntropyLoss()
FineTuning_optimizer=torch.optim.AdamW([{'params':finetuning_model.klue.parameters(), 'lr': 1e-4},
                             {'params':finetuning_model.classifier.parameters(), 'lr': 1e-4}
                             ], weight_decay=0.01)

In [ ]:
FineTuning_train(train_loader, finetuning_model, FineTuning_criterion, FineTuning_optimizer, device, epochs=6)

In [ ]:
torch.save(finetuning_model.state_dict(), save_path + 'phase1_four_label_all_text_finetuning.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=FineTuning_eval(test_loader, finetuning_model, four_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
import gc
del finetuning_model      # 모델 삭제 (변수명 주의)
del FineTuning_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache()
gc.collect()
print("메모리 정리가 완료되었습니다.")

# **six_label_first_text**

In [ ]:
results={'task':'six_label_first_text_mlp',
         'text': 'first',
         'label':'six',
         'model':'mlp'
                    }

In [ ]:
six_label_first_text

In [ ]:
max_length=first_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=MLP_data_load(six_label_first_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mlp_model = MLP_model( hid_dim=512, out_dim=label_count, p=0.3).to(device)

mlp_criterion=nn.CrossEntropyLoss()
mlp_optimizer=torch.optim.AdamW(mlp_model.parameters(),lr=2e-3, weight_decay=1e-4)

In [ ]:
MLP_train(train_loader, mlp_model, mlp_criterion, mlp_optimizer, device, epochs=6)

In [ ]:
torch.save(mlp_model.state_dict(), save_path + 'phase1_six_label_first_text_mlp.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=MLP_eval(test_loader, mlp_model, six_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
import gc
del mlp_model      # 모델 삭제
del mlp_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache() # GPU 캐시 비우기
gc.collect()       # RAM 비우기
print("메모리 정리가 완료되었습니다.")

--------------------------------------------------------------------
##
--------------------------------------------------------------------


In [ ]:
results={'task':'six_label_first_text_finetuning',
         'text': 'first',
         'label':'six',
         'model':'finetuning'
                    }

In [ ]:
max_length=first_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=FineTuning_data_load(six_label_first_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
finetuning_model = klueClassifier(num_class=label_count, num_layers_to_train=3, dropout=0.3).to(device)


FineTuning_criterion=nn.CrossEntropyLoss()
FineTuning_optimizer=torch.optim.AdamW([{'params':finetuning_model.klue.parameters(), 'lr': 1e-4},
                             {'params':finetuning_model.classifier.parameters(), 'lr': 1e-4}
                             ], weight_decay=0.01)

In [ ]:
FineTuning_train(train_loader, finetuning_model, FineTuning_criterion, FineTuning_optimizer, device, epochs=6)

In [ ]:
torch.save(finetuning_model.state_dict(), save_path + 'phase1_six_label_first_text_finetuning.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=FineTuning_eval(test_loader, finetuning_model, six_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
import gc
del finetuning_model      # 모델 삭제 (변수명 주의)
del FineTuning_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache()
gc.collect()
print("메모리 정리가 완료되었습니다.")

# **four_label_first_text**

In [ ]:
results={'task':'four_label_first_text_mlp',
         'text': 'first',
         'label':'four',
         'model':'mlp'
                    }

In [ ]:
four_label_first_text

In [ ]:
max_length=first_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=MLP_data_load(four_label_first_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mlp_model = MLP_model( hid_dim=512, out_dim=label_count, p=0.3).to(device)

mlp_criterion=nn.CrossEntropyLoss()
mlp_optimizer=torch.optim.AdamW(mlp_model.parameters(),lr=2e-3, weight_decay=1e-4)

In [ ]:
MLP_train(train_loader, mlp_model, mlp_criterion, mlp_optimizer, device, epochs=6)

In [ ]:
torch.save(mlp_model.state_dict(), save_path + 'phase1_four_label_first_text_mlp.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=MLP_eval(test_loader, mlp_model, four_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
import gc
del mlp_model      # 모델 삭제
del mlp_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache() # GPU 캐시 비우기
gc.collect()       # RAM 비우기
print("메모리 정리가 완료되었습니다.")

--------------------------------------------------------------------
##
--------------------------------------------------------------------


In [ ]:
results={'task':'four_label_first_text_finetuning',
         'text': 'first',
         'label':'four',
         'model':'finetuning'
                    }

In [ ]:
max_length=first_text_max_length

batch_size=32
epochs=6

In [ ]:
train_loader, test_loader, label_count=FineTuning_data_load(four_label_first_text, max_length)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
finetuning_model = klueClassifier(num_class=label_count, num_layers_to_train=3, dropout=0.3).to(device)


FineTuning_criterion=nn.CrossEntropyLoss()
FineTuning_optimizer=torch.optim.AdamW([{'params':finetuning_model.klue.parameters(), 'lr': 1e-4},
                             {'params':finetuning_model.classifier.parameters(), 'lr': 1e-4}
                             ], weight_decay=0.01)

In [ ]:
FineTuning_train(train_loader, finetuning_model, FineTuning_criterion, FineTuning_optimizer, device, epochs=6)

In [ ]:
torch.save(finetuning_model.state_dict(), save_path + 'phase1_four_label_first_text_finetuning.pt')
print("모델 저장이 완료되었습니다.")

In [ ]:
six_label_depression_classes = [3, 4, 5]
four_label_depression_classes = [1]

In [ ]:
result=FineTuning_eval(test_loader, finetuning_model, four_label_depression_classes,device=device)

In [ ]:
results.update(result)

In [ ]:
total_result.append(results)

In [ ]:
import gc
del finetuning_model      # 모델 삭제 (변수명 주의)
del FineTuning_optimizer  # 옵티마이저 삭제
torch.cuda.empty_cache()
gc.collect()
print("메모리 정리가 완료되었습니다.")

# 비교 및 시각화

In [ ]:
total_result=pd.DataFrame(total_result)

In [ ]:
total_result.to_csv(save_path+'data/phase1_total_result.csv')

In [ ]:
total_result

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
figure, axs = plt.subplots(4,1,figsize=(20,15))

metrics=['accuracy', 'f1_macro', 'depression_precision', 'depression_recall']

for i,v in enumerate(metrics):
    #axs[i].bar(total_result['task'], total_result[v])
    sns.barplot(data=total_result, x='task', y=v, hue='text', ax= axs[i])
    axs[i].set_title(v, fontsize=18)
    axs[i].set_ylabel('Score')
    axs[i].grid(axis='y', alpha=0.3)

plt.tight_layout(h_pad=5.0)

plt.show()

#끝

In [ ]:
figure, axs = plt.subplots(4,1,figsize=(40,30))

metrics=['accuracy', 'f1_macro', 'depression_precision', 'depression_recall']

for i,v in enumerate(metrics):
    #axs[i].bar(total_result['task'], total_result[v])
    sns.barplot(data=total_result, x='task', y=v, hue='text', ax= axs[i])
    axs[i].set_title(v, fontsize=18)
    axs[i].set_ylabel('Score')
    axs[i].grid(axis='y', alpha=0.3)

plt.tight_layout(h_pad=5.0)

plt.show()

In [ ]:
figure, axs = plt.subplots(4,1,figsize=(40,30))

metrics=['accuracy', 'f1_macro', 'depression_precision', 'depression_recall']

for i,v in enumerate(metrics):
    #axs[i].bar(total_result['task'], total_result[v])
    sns.barplot(data=total_result, x='task', y=v, hue='text', ax= axs[i])
    axs[i].set_title(v, fontsize=18)
    axs[i].set_ylabel('Score')
    axs[i].grid(axis='y', alpha=0.3)

plt.tight_layout(h_pad=5.0)

plt.show()